In [2]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    #t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

In [3]:
import os
import time
import pandas as pd

# pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup # pip install beautifulsoup4

import random


Accedemos a la pagina web de Mercadona 

In [4]:
df = pd.DataFrame(columns=['categoria', 'subcategoria', 'descripcion', "unidades", "precio", "code", "compare"])
df

df = pd.DataFrame(columns=['categoria', 'subcategoria', 'descripcion', "unidades", "precio", "code", "compare"])

In [5]:
supermecados = ["mercadona", "dia", "consum"]

#https://tienda.consum.es/es#!Home

In [6]:
supermercado = supermecados[0]

In [ ]:
#ENTRAR EN MERCADONA:

# Start the driver with the options
driver = webdriver.Chrome()

actions = ActionChains(driver)

# Navigate to the login page
link = f'https://www.{supermercado}.es/'
driver.get(link)

#Reject/Accept cookies:
driver.find_element(By.XPATH, "//button[contains(text(),'Rechazar')]").click()

# Enter the postal code and submit
code="08015"

#Find the box elemet for postal code and input the code:
driver.find_element(By.CSS_SELECTOR, 'input[aria-label="Código postal"]').send_keys(code)

# click the botom code 
driver.find_element(By.CSS_SELECTOR, 'input.postal-code-form__button').click()

#Click on Categories Button(Wait til its visible)
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.LINK_TEXT, "Categorías"))).click()

time.sleep(2)

#Save all categories into a variable:
category_links = driver.find_elements(By.CSS_SELECTOR, "span[class='category-menu__header']")

if category_links != 0:
    os.system("say categorias ok, entrando")
else:
    os.system("say categorias vacias")

scrape=0
    
for c in category_links:
    
    
    category = c.text
    #print(category)
    

    
    non = ["Aceite, especias y salsas", "Agua y refrescos", "Aperitivos", "Arroz, legumbres y pasta",
                    "Azúcar, caramelos y chocolate", "Bebé", "Bodega", "Cacao, café e infusiones", "Carne",
                    "Cereales y galletas", "Congelados", "Charcutería y quesos", "Conservas, caldos y cremas",
                    "Cuidado del cabello", "Cuidado facial y corporal", "Fitoterapia y parafarmacia"]
    
    if category in non:
        continue

    
    #Enter the category:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//label[text()='{category}']"))).click()
    
    #time.sleep(random.randint(10,30))
    
    #Target subcategories:
    subcategory_links = driver.find_elements(By.CSS_SELECTOR, ".category-item__link")
    
    if subcategory_links != 0:
        os.system("say subcategorias ok, entrando")
    else:
        os.system("say subcategorias vacias")
        
    #time.sleep()
        
    for s in subcategory_links:
        subcategory = s.text
        
        """
        
        avoid = ["Fruta", "Lechuga y ensalada preparada"]
        
        if subcategory in avoid:
            continue
            
        """
        
        
        print(subcategory)
        #time.sleep(400)
    
        
        #Enter a subcategorie:
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f"//button[text()='{subcategory}']"))).click()
        #print("subclicada")
        
        #time.sleep(1)
        
        # Wait for the products to load and saves them
        WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product-cell")))

        #Extrae todos los productos(cajita que vimos )
        product_cells = driver.find_elements(By.CSS_SELECTOR, "div[data-test='product-cell']")
        
        os.system("say productos ok, entrando")

        for p in product_cells:
            product = p
            #print(product)
            
            #time.sleep(random.randint(2,8))
            
            des = product.find_element(By.CSS_SELECTOR, "h4[data-test='product-cell-name']").text
            unit = product.find_element(By.CSS_SELECTOR, "div[class='product-format product-format__size--cell']").text
            pr = product.find_element(By.CSS_SELECTOR, "div[class='product-price']").text
            
            '--------ANADIMOS URL----------------------------------'
            
            #time.sleep(3) #este yo lo quitaria
            
            # Click on the "product-cell"
            
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "h4[data-test='product-cell-name']")))
            
            actions.move_to_element(product).perform()
            
            product.click()
            scrape += 1
            
            if scrape > 100:
                time.sleep      
                time.sleep(random.randint(300,400))
                scrape = 0

            # Get the current URL of the page
            current_url = driver.current_url
            code = current_url.split("/")[4]
            
            time.sleep(2)
            
            
            WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div[class='product-format product-format__size']")))
     
            info = driver.find_element(By.CSS_SELECTOR, "div[class='product-format product-format__size']").text
            compare = info.split("|")[-1]
            
            # Escape:
            while "categories" not in driver.current_url:
                driver.back()
                driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.ESCAPE)
        
            nueva_fila = {
                'categoria':category , 
                'subcategoria': subcategory, 
                'descripcion': des,
                "unidades": unit,
                "precio":pr,
                "code": code,
                "compare" : compare
                }
            
            # Wait the second half of the time passed before clicking the next product
            #time.sleep(2)
            
            #time.sleep(random.randint(2,10))
  

            df = pd.concat([df, pd.DataFrame([nueva_fila])], ignore_index=True)
            #print("row inserted")
        
    

Fruta
Lechuga y ensalada preparada
Verdura
Huevos
Leche y bebidas vegetales
Mantequilla y margarina
Detergente y suavizante ropa
Estropajo, bayeta y guantes
Insecticida y ambientador
Lejía y líquidos fuertes
Limpiacristales
Limpiahogar y friegasuelos
Limpieza baño y WC
Limpieza cocina
Limpieza muebles y multiusos
Limpieza vajilla
Menaje y conservación de alimentos
Papel higiénico y celulosa
Pilas y bolsas de basura
Utensilios de limpieza y calzado
Bases de maquillaje y corrector
Colorete y polvos
Labios
Ojos
Pinceles y brochas
Marisco
Pescado congelado
Pescado fresco
Salazones y ahumados
Sushi
Gato
Perro
Otros
Bollería de horno
Bollería envasada
Harina y preparado repostería


In [180]:
export_to_csv(df,"mercadona-06")

DataFrame successfully exported to mercadona-06.csv


In [ ]:
scrape